## PromptTemplate

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

### 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체생성


In [4]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

In [7]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울입니다.'

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성

In [8]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [9]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [10]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template = template,
    input_variables=["country1"],
    partial_variables = {
        "country2": "미국"
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [11]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [12]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [14]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [15]:
chain = prompt_partial | llm

In [16]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [17]:
chain.invoke({"country1": "대한민국", "country2":"호주"}).content

'대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.'

### `partial_variables`: 부분 변수 채움

`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것입니다. 이 사용 사례는 **항상 공통된 방식으로 가져오고 싶은 변수** 가 있는 경우입니다.

대표적인 예가 **날짜나 시간** 입니다.

항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해 보겠습니다. 프롬프트에 하드 코딩할 수도 없고, 다른 입력 변수와 함께 전달하는 것도 번거롭습니다. 이 경우 항상 현재 **날짜를 반환하는 함수** 를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리합니다.


In [18]:
from datetime import datetime
datetime.now().strftime("%B %d")

'April 02'

In [19]:
def get_today():
    return datetime.now().strftime("%B %d")

In [20]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today
    },
)

In [21]:
prompt.format(n=3)

'오늘의 날짜는 April 02 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [22]:
chain = prompt | llm

In [23]:
print(chain.invoke(3).content)

1. 방탄소년단 뷔 (김태형) - 1995년 12월 30일
2. 배우 이수혁 - 1988년 4월 2일
3. 싱어송 라이터 루카스 그레이엄 - 1993년 4월 2일


In [24]:
print(chain.invoke({"today":"Jan 02", "n":3}))

content='1. 김태희 (1980년 3월 29일)\n2. 조정석 (1980년 3월 11일)\n3. 브리트니 스피어스 (1981년 12월 2일)' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 55, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BHlECFblGNDoPLoT2eIZsvcUWcWnH', 'finish_reason': 'stop', 'logprobs': None} id='run-e240fb43-9957-45c2-81aa-4743aaa4276f-0' usage_metadata={'input_tokens': 55, 'output_tokens': 69, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## 파일로부터 template 읽어오기

In [31]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [32]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [34]:
prompt2 = load_prompt("prompts/capital.yaml", encoding="utf-8")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



## ChatPromptTemplate

`ChatPromptTemplate` 은 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

메시지는 튜플(tuple) 형식으로 구성하며, (`role`, `message`) 로 구성하여 리스트로 생성할 수 있습니다.

**role**
- `"system"`: 시스템 설정 메시지 입니다. 주로 전역설정과 관련된 프롬프트입니다.
- `"human"` : 사용자 입력 메시지 입니다.
- `"ai"`: AI 의 답변 메시지입니다.

In [35]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [36]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [37]:
from langchain_core.prompts import  ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
    
)

messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)

messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [38]:
llm.invoke(messages).content

'제 이름은 테디입니다. 궁금한 점이 있으시면 언제든지 물어보세요!'

In [39]:
chain = chat_template | llm

In [40]:
chain.invoke({"name":"Teddy", "user_input":"당신의 이름은 무엇입니까?"}).content

'제 이름은 Teddy입니다. 당신을 도와드릴 수 있어요!'

## MessagePlaceholder

또한 LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 `MessagePlaceholder` 를 제공합니다. 

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [41]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [42]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [43]:
chain = chat_prompt | llm | StrOutputParser()

In [44]:
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'테디: 새로 입사함, 만나서 반가움.'